In [54]:
import pandas as pd
import numpy as np
import os
import pickle
from fuzzywuzzy import fuzz

c:\Users\prati\anaconda3\envs\DataGUI\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [50]:
hsn_code_data = pd.read_csv('hsn_code_des.csv')

In [47]:
statista_data = os.listdir('statista_data_csv')

In [45]:
# amazon_category_list = {
#     'Beauty & Personal Care':{
#     'Makeup':['Body Makeup', 'Eye Makeup', 'Face Makeup', 'Lip Makeup', 'Makeup Palettes', 'Makeup Remover', 'Makeup Sets'],
#     'Skin Care':['Body','Eyes','Face','Lip Care','Maternity','Sets & Kits','Sunscreens & Tanning Products'],
#     'Hair Care':['Hair Styling Accessories','Hair Coloring Products','Hair Cutting Tools','Hair Extensions, Wigs & Accessories',
#     'Hair Loss','Hair Treatment Masks','Hair Perms, Relaxers & Texturizers','Hair Treatment Oils','Scalp Treatments',
#     'Shampoo & Conditioner','Hair Styling Products','Styling Tools & Appliances'],
#     'Perfumes & Fragrances':['Children\'s Fragrance','Fragrance Dusting Powders','Men\'s Fragrances','Fragrance Sets','Women\'s Fragrances'],
#     'Foot, Hand & Nail Care':['Foot & Hand Care','Nail Polish & Decoration Products','Nail Care Products','Hand, Foot & Nail Tools'],
#     'Beauty Tools & Accessories':['Makeup Bags & Cases','Bathing Accessories','Cotton Balls & Swabs','Makeup Brushes & Tools',
#     'Personal Mirrors','Refillable Cosmetic Containers','Shaving & Hair Removal Products','Skin Care Tools'],
#     'Shaving & Hair Removal':['Men\'s Shaving & Hair Removal Products','Women\'s Shaving & Hair Removal Products'],
#     'Personal Care':['Bath & Bathing Accessories','Deodorants & Antiperspirants','Lip Care Products','Oral Care Products',
#     'Piercing & Tattoo Supplies','Body Scrubs & Treatments','Shaving & Hair Removal Products','Skin Care Products'],
#     'Salon & Spa Equipment':['Hair Drying Hoods','Galvanic Facial Machines','Handheld Mirrors','High Frequency Facial Machines',
#     'Manicure Tables','Professional Massage Equipment','Salon & Spa Stools','Spa Beds & Tables','Salon & Spa Chairs',
#     'Spa Storage Systems','Spa Hot Towel Warmers']
#     },
#     'Grocery & Gourmet Food':{
#         'Alcoholic Beverages':['Beer','Cocktail Mixers','Liquor & Spirits','Wine'],
#         'Baby Foods':['Baby Beverages','Cereal & Porridge for Babies','Baby & Toddler Formula','Baby Food Meals','Baby Snack Foods'],
#         'Beverages':['Beverage Syrups & Concentrates','Bottled Beverages, Water & Drink Mixes','Bubble Tea Products','Coffee',
#         'Coffee Gifts','Coffee Substitutes','Hot Chocolate & Malted Drinks','Infusion Mixers for Alcohol & Spirits','Tea Beverages'],
#         'Breads & Bakery':['Bakery & Dessert Gifts','Fresh Bakery Breadcrumbs & Stuffing','Breads','Fresh Bakery Breadsticks','Cakes',
#         'Grocery Cookies','Bakery Desserts','Breakfast Bakery','Pastry Shells & Crusts','Pizza Crusts','Refrigerated Doughs'],
#         'Breakfast Foods':['Applesauce & Fruit Cups','Breakfast & Cereal Bars','Breakfast Syrups & Toppings','Breakfast Cereals','Instant Breakfast Drinks',
#         'Breakfast Pancake, Waffle & Baking Mixes','Toaster Pastries'],
#         'Dairy, Eggs & Plant-Based Alternatives':['Butter, Margarine & Plant-Based Alternatives','Cheeses','Cheese & Charcuterie Gifts','Cheese Assortments & Samplers',
#         'Refrigerated Cheese Dips & Spreads','Cottage Cheeses','Cream Cheeses','Dairy & Plant-Based Yogurt','Eggs & Egg Substitutes','Milks & Creams','Plant-Based Cheese',
#         'Plant-Based Coffee Creamers','Plant-Based Milk','Refrigerated Snack Packs & Lunches','Sour Creams','Whipped Toppings'],
#         'Deli & Prepared Foods':['Fresh Prepared Appetizers','Fresh Prepared Box Lunches','Fresh Prepared Breakfast Foods','Fresh Prepared Casseroles, Pot Pies & Quiches',
#         'Deli Meats & Cheeses','Fresh Prepared Deli Salads','Fresh Prepared Dips, Salsas & Spreads','Fresh Prepared Entrees','Fresh Prepared Pasta & Sauces',
#         'Fresh Prepared Risotto','Fresh Meal Ingredient Kits','Antipasto','Fresh Prepared Sandwiches & Wraps','Fresh Prepared Side Dishes','Fresh Prepared Soups, Stews & Chili'],
#         'Food & Beverage Gifts':['Food & Drink Advent Calendars','Food Assortments & Variety Gifts','Bakery & Dessert Gifts',
#         'Candy & Chocolate Gifts','Cheese & Charcuterie Gifts','Coffee Gifts','Fruit & Nut Gifts','Herb, Spice & Seasoning Gifts',
#         'Jam, Jelly & Sweet Spread Gifts','Meat & Seafood Gifts','Sauce, Gravy & Marinade Gifts','Snack Food Gifts','Gourmet Tea Gifts'],
#         'Fresh Flowers & Live Indoor Plants':['Fresh Flower & Plant Centerpieces','Fresh Cut Flowers','Fresh Flower & Plant Sprays','Fresh Flower & Plant Wreaths','Live Indoor House Plants'],
#         'Fresh Meal Ingredient Kits':[],
#         'Frozen Foods':['Frozen Appetizers & Snacks','Frozen Bread & Dough','Frozen Breakfast Foods','Frozen Desserts & Toppings','Frozen Fruits',
#         'Ice Creams & Frozen Novelties','Frozen Juices','Frozen Meals & Entrees','Frozen Meat','Frozen Pasta & Sauces','Frozen Pizzas','Frozen Potatoes & Onion Rings',
#         'Frozen Seafood','Frozen Vegetables'],
#         'Home Brewing & Winemaking':['Beer Brewing Ingredients','Beer Brewing Recipe Kits''Winemaking Ingredients'],
#         'Meat & Seafood':['Bacon','Beef','Chicken','Foie Gras & Pâtés','Frozen Meat','Hot Dogs & Franks','Lamb Meat','Meat & Seafood Gifts','Pork Meat',
#         'Sausages','Seafood','Turducken','Turkey','Veal Meats','Wild Game & Fowl Meat'],
#         'Meat Substitutes':['Meatless Bacon','Meatless Burgers & Patties','Meatless Deli Slices''Meatless Hot Dogs, Links & Sausages','Meatless Jerky',
#         'Meatless Loaves & Roasts','Meatless Meatballs, Chunks & Crumbles','Meatless Nuggets & Wings','Meatless Pulled & Flaked Meats','Fresh Seitan',
#         'Meatless Strips & Cutlets','Fresh Tempeh','Fresh Tofu'],
#         'Pantry Staples':['Canned, Jarred & Packaged Foods','Condiments & Salad Dressings','Cooking & Baking Grocery Supplies','Dried Beans, Lentils & Peas',
#         'Dried Grains & Rice','Herbs, Spices & Seasonings','Jams, Jellies & Sweet Spreads','Nut & Seed Butters','Olives, Pickles & Relishes','Pasta & Noodles',
#         'Sauces, Gravies & Marinades','Soups, Stocks & Broths'],
#         'Fresh Produce':['Dried Fruits & Vegetables','Fresh Fruits','Fresh Herbs','Fresh Vegetables','Fruit & Nut Gifts','Cooking & Baking Nuts & Seeds',
#         'Fresh Packaged Produce','Fresh Vegetarian Proteins'],
#         'Snacks & Sweets':['Candy & Chocolate','Chocolate','Snack Foods']
#     }
# }

# with open('amazon_category_list.pickle', 'wb') as handle:
#     pickle.dump(amazon_category_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [52]:
amazon_category_list.keys()

dict_keys(['Beauty & Personal Care', 'Grocery & Gourmet Food'])

In [59]:
hsn_code_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9323 entries, 0 to 9322
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         9323 non-null   int64  
 1   HSN_code           8916 non-null   object 
 2   name_of_commodity  8917 non-null   object 
 3   chapter_no         8909 non-null   float64
 4   sch                8910 non-null   object 
 5   gst                8887 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 437.1+ KB


In [61]:
hsn_code_data['name_of_commodity'].fillna('NA',inplace=True)

In [73]:
hsn_mapping = pd.DataFrame()
for key in amazon_category_list.keys():
    for k in key.split(' & '):
    hsn_mapping['mapped_score_'+key] = hsn_code_data['name_of_commodity'].apply(lambda x: fuzz.partial_ratio(key,x))

In [74]:
hsn_mapping['mapped_score_Beauty & Personal Care'].argmax()

2810

In [76]:
hsn_mapping['mapped_score_'+'Body Makeup'] = hsn_code_data['name_of_commodity'].apply(lambda x: fuzz.partial_('Body Makeup',x))

In [77]:
hsn_mapping['mapped_score_'+'Body Makeup'].argmax()

600

In [79]:
hsn_code_data.iloc[600,:]

Unnamed: 0                                    601
HSN_code                               0903.00.00
name_of_commodity                            Mate
chapter_no                                    9.0
sch                                            II
gst                                            5%
mapped_score_Beauty & Personal Care             8
mapped_score_Grocery & Gourmet Food             8
Name: 600, dtype: object

In [72]:
hsn_code_data[hsn_code_data['name_of_commodity'].str.contains('Beauty')]

,Unnamed: 0,HSN_code,name_of_commodity,chapter_no,sch,gst,mapped_score_Beauty & Personal Care,mapped_score_Grocery & Gourmet Food
3543,3544,3304.00.00,Beauty or make-up preparations and\npreparatio...,33.0,V,28%,17,13
3548,3549,3304.91.00,Beauty or make-up preparations (powder type),33.0,V,28%,36,18
